# Get List of articles usnig crossref api

### import the required libraries

In [9]:
import requests
import pandas as pd
from time import time

### Set API address and parameters

In [10]:
api_url = "https://api.crossref.org/works"
query_params = {
    "query": "LiDAR SLAM",
    "rows": 1000,  # Adjust based on how many results you want per page
    # "offset": 0, # Deprecated use cursor instead
    "cursor": "*"  # Starting cursor
}

### Make test request

In [11]:
test_params = query_params.copy()
test_params["rows"] = 10

response = requests.get(api_url, params=test_params)
if response.status_code != 200: raise SystemExit("Bad response")

data = response.json()
Number_of_articles = data['message']['total-results']
print(f"Query: {data['message']['query']}")
print(f"The number of work matched criteria = {Number_of_articles}\n")

Query: {'start-index': 0, 'search-terms': 'LiDAR SLAM'}
The number of work matched criteria = 46913



### Function to get information about the article

In [4]:
import unicodedata

def preprocess_text(text):
    text = text.replace("\n", " ") # remove new lines
    text = text.replace('/',' / ') # ensure space between words
    text = text.replace(",", ", ").replace(",  ", ", ") # replace commas without space
    return unicodedata.normalize('NFKD', text) 

def get_article_info(item):
    Authors = item.get('author', [])
    article_info = {
        "title": preprocess_text(item.get('title', ['No Title'])[0]),
        "DOI": item.get('DOI'),
        "publisher": item.get('publisher'),
        "publisher_location": item.get('publisher-location'),
        "year": item.get('created', {}).get('date-parts', [[None]])[0][0],
        "type": item.get('type'),
        "score": item.get('type'),
        "reference-count": item.get('reference-count'),
        "is-referenced-by-count": item.get('is-referenced-by-count'),
        "authors": ', '.join([author.get('given', '') + " " + author.get('family', '') for author in Authors])
    }
    
    try: # use try because some fields may be empty
        first_author_affiliation = Authors[0].get('affiliation', [])
        affiliation_location = first_author_affiliation[0].get('name')
        article_info["publisher_affiliation"] = preprocess_text(affiliation_location)
    except Exception:
        article_info["publisher_affiliation"] = None
    
    return article_info

### Getting the list of articles and saving them into an array 

In [5]:
cur_query_params = query_params.copy()
articles = []  # List to store article information

start = time()
while True: 
    # make request
    response = requests.get(api_url, params=cur_query_params) 
    if response.status_code != 200: raise SystemExit(f"Bad response ({response.status_code}): {response.json()["message"]}")

    # read list of works    
    data = response.json()
    items = data['message']['items']

    # stop if no work remains
    if not items: break

    # get article info
    for item in items:
        articles.append(get_article_info(item))

    cur_query_params['cursor'] = data['message'].get('next-cursor')
    print(f"articles found: {len(articles)}/{Number_of_articles} ({(time() - start):.2f} sec)")

articles found: 1000/46913 (3.16 sec)
articles found: 2000/46913 (6.43 sec)
articles found: 3000/46913 (9.02 sec)
articles found: 4000/46913 (12.22 sec)
articles found: 5000/46913 (14.89 sec)
articles found: 6000/46913 (18.37 sec)
articles found: 7000/46913 (21.27 sec)
articles found: 8000/46913 (34.42 sec)
articles found: 9000/46913 (41.55 sec)
articles found: 10000/46913 (47.80 sec)
articles found: 11000/46913 (54.70 sec)
articles found: 12000/46913 (66.17 sec)
articles found: 13000/46913 (72.55 sec)
articles found: 14000/46913 (78.27 sec)
articles found: 15000/46913 (84.59 sec)
articles found: 16000/46913 (89.01 sec)
articles found: 17000/46913 (93.53 sec)
articles found: 18000/46913 (100.57 sec)
articles found: 19000/46913 (106.23 sec)
articles found: 20000/46913 (111.57 sec)
articles found: 21000/46913 (121.10 sec)
articles found: 22000/46913 (128.15 sec)
articles found: 23000/46913 (134.22 sec)
articles found: 24000/46913 (140.89 sec)
articles found: 25000/46913 (149.50 sec)
arti

### Create pandas dataframe 

In [6]:
df = pd.DataFrame(articles)
df.head()

,title,DOI,publisher,publisher_location,year,type,score,reference-count,is-referenced-by-count,authors,publisher_affiliation
0,D3VIL-SLAM: 3D Visual Inertial LiDAR SLAM for ...,10.1109/iv55152.2023.10186534,IEEE,None,2023,proceedings-article,proceedings-article,23.0,0.0,"Matteo Frosi, Matteo Matteucci",Information and Bioengineering Politecnico di ...
1,Visual and lidar-based SLAM by variational bay...,10.32657/10356/139813,Nanyang Technological University,None,2020,dissertation,dissertation,0.0,0.0,", Xiaoyue Jiang",None
2,A Comparison of Outdoor 3D Reconstruction betw...,10.1109/cacs60074.2023.10325866,IEEE,None,2023,proceedings-article,proceedings-article,10.0,0.0,"Yi-Tian Hong, Han-Pang Huang","National Taiwan University, Department of Mech..."
3,Indoor mapping and positioning applications of...,10.51946/melid.927004,Turkiye lidar dergisi (Mersin University),None,2021,journal-article,journal-article,29.0,5.0,Mustafa ZEYBEK,None
4,SC-LiDAR-SLAM: A Front-end Agnostic Versatile ...,10.1109/iceic54506.2022.9748644,IEEE,None,2022,proceedings-article,proceedings-article,46.0,10.0,"Giseop Kim, Seungsang Yun, Jeongyun Kim, Ayoun...","KAIST, Dept. of Civil and Envtl. Eng., Daejeon..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46913 entries, 0 to 46912
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   title                   46913 non-null  object 
 1   DOI                     46913 non-null  object 
 2   publisher               46913 non-null  object 
 3   publisher_location      3887 non-null   object 
 4   year                    46913 non-null  int64  
 5   type                    46911 non-null  object 
 6   score                   46911 non-null  object 
 7   reference-count         46878 non-null  float64
 8   is-referenced-by-count  46878 non-null  float64
 9   authors                 46913 non-null  object 
 10  publisher_affiliation   9151 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 3.9+ MB


### Save data

In [8]:
# Save the DataFrame to a CSV file with proper handling of newline characters
df.to_csv('slam_articles.csv', index=False, encoding='utf-8', quoting=1)